# Using Spectral Weights

This guide shows how to weight Scans and spectra during averaging and how to find out what weights were used in the process.


You can find a copy of this tutorial as a Jupyter notebook [here](https://github.com/GreenBankObservatory/dysh/blob/main/notebooks/examples/using_spectral_weights.ipynb) or download it by right clicking  <a href="https://raw.githubusercontent.com/GreenBankObservatory/dysh/refs/heads/main/notebooks/examples/using_spectral_weights.ipynb" download>here</a> and selecting "Save Link As".

## Loading Modules
We start by loading the modules we will use for the data reduction. 

For display purposes, we use the static (non-interactive) matplotlib backend in this tutorial. However, you can tell `matplotlib` to use the `ipympl` backend to enable interactive plots. This is only needed if working on jupyter lab or notebook.

In [ ]:
# Set interactive plots in jupyter.
#%matplotlib ipympl

# These modules are required for the data reduction.
from dysh.fits.gbtfitsload import GBTFITSLoad

# This module is used for custom plotting.
import matplotlib.pyplot as plt

# These modules are only used to download the data.
from pathlib import Path
from dysh.util.download import from_url
import numpy as np
import astropy.units as u
np.set_printoptions(precision=4, threshold=11, floatmode='fixed')

## Data Retrieval

Download the example SDFITS data, if necessary.

In [ ]:
#url = "http://www.gb.nrao.edu/dysh/example_data/mixed-fs-ps/data/TGBT24B_613_04.raw.vegas.trim.fits"
#savepath = Path.cwd() / "data"
#savepath.mkdir(exist_ok=True) # Create the data directory if it does not exist.
#filename = from_url(url, savepath)

In [ ]:
from dysh.util import get_project_testdata
filename = get_project_testdata() / "AGBT05B_047_01/AGBT05B_047_01.raw.acs"

## Data Loading

Next, we use `GBTFITSLoad` to load the data, and then its `summary` method to inspect its contents.

In [ ]:
sdf = GBTFITSLoad(filename)
sdf.summary()

### Position Switched Calibration
We calibrate several scans at once for NGC5291.  This results in a ScanBlock with 4 scans, each with 11 integrations. The default weights are unity, with one weight value per integration.


In [ ]:
pssb = sdf.getps(object='NGC5291',ifnum=0,plnum=1,fdnum=0)
print(f"Number of scans: {len(pssb)}, Number of integrations per scan: {[len(k) for k in pssb]}")
print(f"Scan weights: {pssb.weights}")

## Applying weights when averaging Scans
When averaging scans to create a final spectrum, you can choose from dysh's two options ('tsys' or None) or supply your own weight array.

### 1. System temperature weighting
The default for `timeaverage()` is 'tsys' which calculates and applies system temperature weighting to each integration: 

$w = t_{exp} \times \delta\nu/T_{sys}^2$,

where $t_{exp}$ is the integraton exposure time, $\delta\nu$ is the channel width, and $T_{sys}$ is the system temperature.  You can check what those weights will be with `tsys_weight`. 

In [ ]:
pssb.tsys_weight

(Note this is an array of arrays rather than a multidimensional array because different Scans may have differing numbers of integrations).

`timeaverage()` with the default arguments will use these weights.  The final spectral weight is the sum of all weights.

In [ ]:
ta = pssb.timeaverage()  # default is weights='tsys'
exclude_regions=[(1.37*u.GHz,1.38*u.GHz),(1.395*u.GHz,1.405*u.GHz),(1.42*u.GHz,1.43*u.GHz)]
ta.baseline(degree=1, remove=True, exclude=exclude_regions)
ta.plot(ymin=-0.25,ymax=0.3)
print(f"final weights={ta.weights}")
ta.stats()

### 2. Equal weighting
Supplying `weights=None` will weight all integrations the same.  In this case, the result is not much different thatn `tsys`, because the $T_{sys}$ weights were already pretty uniform.

In [ ]:
ta2 = pssb.timeaverage(weights=None)  
ta2.baseline(degree=1,remove=True,exclude=exclude_regions)
ta2.plot(ymin=-0.25,ymax=0.3)
print(f"final weights={ta2.weights}")
ta2.stats()

### 3. User-supplied weights
You can supply a numpy array of weights to apply.  For `ScanBlock.timeavearage()` the weights must have shape `(Nint,)` or `(Nint,nchan)` where `Nint` is the number of integrations in the ScanBlock and `nchan` is the number of channels in each Scan.  

#### 3a.  Number of weights equal to number of integrations.
We create a slightly silly example, that weights later integrations more.

In [ ]:
w = np.arange(1,pssb.nint+1,dtype=float)
ta3 = pssb.timeaverage(weights=w)
ta3.baseline(degree=1,remove=True,exclude=exclude_regions)
ta3.plot(ymin=-0.25,ymax=0.3)
print(f"final weights={ta3.weights}")

#### 3b.  Weights array with shape `(nint,nchan)`
Supposed you had a channel-based $T_{sys}$ for each integration and wanted to calculate and apply system temperature weights. This can be accomplished by giving a weights array to `ScanBlock.timeaverage()`.

First we fake a `tsys` array of the correct shape.  Then we calculate system temperate weights using the mean exposure time and mean channel width of the Scans.

In [ ]:
from dysh.spectra import tsys_weight
tsys=30+np.random.rand(pssb.nint,pssb.nchan)*15.0  # a fake system temperature array in the 30-45K range
dt = np.mean(np.mean(pssb.exposure))
df = np.mean(np.mean(pssb.delta_freq))
w=tsys_weight(dt,df,tsys)
print(f"{tsys=}")
print(f"Weights array shape: {w.shape}")

In [ ]:
ta4 = pssb.timeaverage(weights=w)
ta4.baseline(degree=1,remove=True,exclude=exclude_regions)
ta4.plot(ymin=-0.25,ymax=0.3)
print(f"final weights={ta4.weights}")

### 4. Averaging lists of spectra with pre-computed weights
The method `average_spectra` can compute weighted averages in 3 ways.  The first two are the usual `weights='tsys'` and `weights=None` options.  The third option, `weights='spectral'` will average the spectra using the values in each of their `weights` array. 

In [ ]:
from dysh.spectra import average_spectra 
sp = average_spectra([ta,ta2,ta3,ta3],weights='spectral')

In [ ]:
sp.plot(ymin=-0.25,ymax=0.3)
print(f"final weights={sp.weights}")